# Generating IC's

In [6]:
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
import os
import pyccl as ccl

We start by generating some white noise $n(\mathbf{x})$

In [7]:
size = 30 # Mpc/h
n_grid = 128
n = jax.random.normal(jax.random.PRNGKey(0), (n_grid, n_grid, n_grid))

In [9]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05, h=0.7, A_s=2.1e-9, n_s=0.96, Neff=0, Omega_g=0)

# compute the power spectrum
ell = jnp.arange(1, 10000)
cl = ccl.angular_cl(cosmo, ell)
cl = jnp.array(cl)

# compute the 3D power spectrum
k = jnp.fft.fftfreq(n_grid, size/n_grid)
k = jnp.sqrt(jnp.sum(jnp.square(k), axis=0))
pk = ccl.nonlin_matter_power(cosmo, k)
pk = jnp.array(pk)

# compute the 3D field
def get_field(n, pk):
    n = jnp.fft.fftn(n)
    n = jnp.fft.ifftn(n * jnp.sqrt(pk))
    return n

field = get_field(n, pk)

# compute the 2D field
def get_2d_field(field):
    field = jnp.fft.fft2(field)
    field = jnp.fft.fftshift(field)
    field = jnp.abs(field)
    return field

field_2d = get_2d_field(field)

# plot the 2D field
plt.imshow(field_2d)


TypeError: angular_cl() missing 2 required positional arguments: 'tracer2' and 'ell'